In [1]:
import os
import sys
import pandas as pd
import numpy as np
import shapefile as shp
import osmnx as ox
import contextily as ctx
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D  # for legend handle
from matplotlib_scalebar.scalebar import ScaleBar
from sklearn.metrics.pairwise import haversine_distances
from shapely.geometry import Point
from pyproj import Proj, transform
import math
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import hashlib
from datetime import datetime, timedelta

In [2]:
import sys
base_directory = "/Users/johnbarrera/Documents/Projects/world_bank/Climate-and-Disaster-Risk-Management-for-Health-Systems/"
sys.path.append(f"{base_directory}")


from src.utils.config_reader import ConfigReader

config_reader = ConfigReader()
config_file_path = "configs/nepal/data_setup.yaml"
config_data = config_reader.read_configuration_file(f"{base_directory}{config_file_path}")
config_data

FileNotFoundError: [Errno 2] No such file or directory: '/Users/johnbarrera/Documents/Projects/world_bank/Climate-and-Disaster-Risk-Management-for-Health-Systems/configs/nepal/data_setup.yaml'

In [ ]:
population_exposure_config = None
for process in config_data['processing']:
    if process['type'] == 'Population Exposure Calculation':
        population_exposure_config = process

df_population = pd.read_csv(f"{base_directory}{population_exposure_config['population_source']}")

df_population

In [ ]:
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "/Users/johnbarrera/Documents/Projects/world_bank/Climate-and-Disaster-Risk-Management-for-Health-Systems"))


In [ ]:
import rasterio
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import geopandas as gpd
from tqdm import tqdm


In [ ]:
input_path = "../data/nepal/inputs/pga_475y.tif"
input_path2 = "../data/nepal/inputs/pga_specifications.xlsx"
# input_path2 = "../data/nepal/inputs/pga_specifications.tsv"
# input_path2 = "../data/nepal/inputs/pga_specifications.csv"
output_path = "../data/nepal/outputs/map/Peak_Ground_Acceleration.gpkg"


In [ ]:
# Abre el archivo TIF
with rasterio.open(input_path) as src:
    # Imprime información sobre el archivo
    print(src.profile)

    # Lee todas las bandas y guarda los valores en una matriz
    data = src.read()

    # Imprime el número de bandas y el tamaño de la matriz
    print(f'Número de bandas: {src.count}')
    print(f'Tamaño de la matriz: {data.shape}')



In [ ]:
# Definir una función para transformar las coordenadas de los píxeles a coordenadas de mapa
def pixel_to_map_coordinates(transform, col, row):
    x, y = transform * (col, row)
    return x, y

# Abrir el archivo TIF
with rasterio.open(input_path) as src:
    # Leer todas las bandas y guardar los valores en una matriz
    data = src.read()

    # Obtener la transformación de coordenadas de píxeles a coordenadas de mapa
    transform = src.transform

    # Crear una lista vacía para almacenar los datos de cada polígono
    polygons = []

    # Iterar sobre cada banda y crear polígonos para cada píxel con valor distinto de cero
    for i in range(src.count):
        band_data = data[i, :, :]

        for row in tqdm(range(band_data.shape[0])):
            for col in range(band_data.shape[1]):
                # Obtener el valor del píxel
                value = band_data[row, col]

                # Si el valor es cero, ignorar el píxel
                if value > 0:
                    #continue

                    # Calcular las coordenadas de los cuatro vértices del polígono
                    x1, y1 = pixel_to_map_coordinates(transform, col, row)
                    x2, y2 = pixel_to_map_coordinates(transform, col + 1, row)
                    x3, y3 = pixel_to_map_coordinates(transform, col + 1, row + 1)
                    x4, y4 = pixel_to_map_coordinates(transform, col, row + 1)

                    # Crear el polígono a partir de los vértices
                    poly = Polygon([(x1, y1), (x2, y2), (x3, y3), (x4, y4)])

                    # Agregar el polígono y su información a la lista de polígonos
                    polygon_data = {
                        'band': i+1,
                        'value': value,
                        'geometry': poly
                    }
                    polygons.append(polygon_data)
                    
# Crear un GeoDataFrame a partir de la lista de polígonos
gdf = gpd.GeoDataFrame(polygons)

# Agregar el polígono y su información
pga_specifications = pd.read_csv(f"{input_path2}", delimiter='\t')

gdf.to_file(output_path, driver="GPKG")

In [ ]:
# Plot the GeoDataFrame with colors based on the 'value' column
gdf.plot(column='value', cmap='viridis', legend=True, figsize=(10, 10))

# Add title and labels
plt.title('Map with Colors per Value Column')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.show()

In [ ]:
# pga_specifications = pd.read_csv(f"{input_path2}", delimiter='\t')

pga_specifications = pd.read_excel(f"{input_path2}")  

# dtype_mapping = {
#     'Acceleration_min': float,
#     'Acceleration_max': float, 
# }

# pga_specifications = pd.read_csv(f"{input_path2}", dtype=dtype_mapping)
pga_specifications

In [ ]:
pga_specifications.dtypes

In [ ]:
gdf

In [ ]:
# Instrumental_Intensity             I
# Acceleration_g            < 0.000464
# Acceleration_min                -1.0
# Acceleration_max            0.000464
# Velocity_cmxs               < 0.0215
# Perceived_shaking           Not felt
# Potential_damage                 NaN
# Name: 0, dtype: object

def identify_interval(value, df, min_col, max_col):
    for index, row in df.iterrows():
        min_value = row[min_col]
        max_value = row[max_col]
        if min_value <= value and value < max_value:
            return row['Instrumental_Intensity'], row['Acceleration_g'], row['Velocity_cmxs'], row['Perceived_shaking'], row['Potential_damage']
    
    

def gdf_interval_df(gdf, df, value_col, min_col, max_col):
    '''
    '''
    gdf2 = gdf.copy()
    resultado = gdf2.apply(lambda x: identify_interval(x[value_col], df, min_col, max_col), axis=1)
    gdf2['Instrumental_Intensity'] = [x[0] for x in resultado]
    gdf2['Acceleration_g'] = [x[1] for x in resultado]
    gdf2['Velocity_cmxs'] = [x[2] for x in resultado]
    gdf2['Perceived_shaking'] = [x[3] for x in resultado]
    gdf2['Potential_damage'] = [x[4] for x in resultado]

    return gdf2

gdf2 = gdf_interval_df(gdf, pga_specifications, value_col='value', min_col='Acceleration_min', max_col='Acceleration_max')
gdf2

In [ ]:
set(gdf2['Instrumental_Intensity'])

In [ ]:
assets_geodataframe: comenzar con hospitales y earthquake

para hospital: 
        id 
        nombre 
        geometria
        costo de reparacion
        recontruccion coste
        admin level



In [ ]:
# # 
# def exposure (hazards_geodataframe, assets_geodataframe, substantial_damage, complete_destruction=None):
#     '''
#     hazards_geodataframe: 
#     assets_geodataframe: dataframde infraestucturas puntos geometricos, lineas, o polygonos (son las infraestruturas)
#     '''
    
#     # if complete_destruction
#         # ver que polygonos:  substacia_damage <= X < complete_destruction
#             hazards_geodataframe_filtrado_substantial_damage
    
#             hazards_geodataframe_filtrado_complete_destruction (# >= complete_destruction)
#     # else
#         # ver que polygonos estan dentro de substacia damage (>=)
        
#         hazards_geodataframe_filtrado_substantial_damage
        
    
#     assets_geodataframe:    damage_level (0.5, or 1) 
        
    
#     return 
